In [5]:
import pandas as pd
from datetime import datetime, timedelta
from pandas.api.types import CategoricalDtype

# chained_assignment = 한줄에 여러 동작을 하는 코드
# chained_assignment에 대해 pandas가 어떻게 대응할지를 설정
pd.set_option('mode.chained_assignment',  None)

# 1. covid19 원본 데이터셋 로딩
# covid19 데이터 로딩(파일을 다운로드 받은 경우)
df_covid19 = pd.read_csv("owid-covid-data.csv")
df_covid19.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [6]:
## df_covid19['date']를 datetime으로 변환
# After this conversion, the 'date' column will contain datetime objects instead of strings, 
# which allows for easier manipulation and analysis of date-related data.
# print(type(df_covid19['date'][0])) # -> str
df_covid19['date'] = pd.to_datetime(df_covid19['date'], format="%Y-%m-%d")
# print(type(df_covid19['date'][0])) # -> timestamp

In [7]:
## 대륙 데이터와 최종 데이터로부터 100일 전 데이터 필터링
df_covid19_100 = df_covid19[(df_covid19['iso_code'].isin(['KOR', 'OWID_ASI', 'OWID_EUR', 
'OWID_OCE', 'OWID_NAM', 'OWID_SAM', 'OWID_AFR']))  # isin = 뒤에 있는 문자열이 포함된 행을 TRUE로 표시한 시리즈를 반환
& (df_covid19['date'] >= (max(df_covid19['date']) - timedelta(days = 100)))] # date가 최근 100일이면

In [8]:
## 대륙명을 한글로 변환
# location = country인 마스크로 불리언 시리즈 반환,
# location 열에 있는 위의 조건이 True인 곳을 한국어로 변경
df_covid19_100.loc[df_covid19_100['location'] == 'South Korea', "location"] = '한국'
df_covid19_100.loc[df_covid19_100['location'] == 'Asia', "location"] = '아시아'
df_covid19_100.loc[df_covid19_100['location'] == 'Europe', "location"] = '유럽'
df_covid19_100.loc[df_covid19_100['location'] == 'Oceania', "location"] = '오세아니아'
df_covid19_100.loc[df_covid19_100['location'] == 'North America', "location"] = '북미'
df_covid19_100.loc[df_covid19_100['location'] == 'South America', "location"] = '남미'
df_covid19_100.loc[df_covid19_100['location'] == 'Africa', "location"] = '아프리카'

In [9]:
## 이산형 변수 (뚝뚝 끊어지는 변수 ex. 코로나 감염자는 하루마다 몇명씩 정해져있음) 설정
ord = CategoricalDtype(categories = ['한국', '아시아', '유럽', '북미', '남미', 
'아프리카','오세아니아'], ordered = True)

df_covid19_100['location'] = df_covid19_100['location'].astype(ord)
# This categorical data type can be used to assign categorical values to a Pandas DataFrame column, 
# allowing for efficient storage and manipulation of data with a limited number of distinct values. Additionally, 
# the ordering of the categories can be useful for performing operations such as sorting and comparisons.

In [10]:
## date로 정렬
df_covid19_100 = df_covid19_100.sort_values(by = 'date')
df_covid19_100.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
2016,OWID_AFR,NaN,아프리카,2022-10-03,12358785.0,754.0,907.714,256770.0,6.0,5.714,...,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN,NaN
163005,OWID_NAM,NaN,북미,2022-10-03,114321967.0,45451.0,52119.000,1513772.0,331.0,505.429,...,NaN,NaN,NaN,NaN,NaN,6.003237e+08,NaN,NaN,NaN,NaN
207753,OWID_SAM,NaN,남미,2022-10-03,63998029.0,15450.0,11810.571,1329632.0,375.0,123.429,...,NaN,NaN,NaN,NaN,NaN,4.368167e+08,NaN,NaN,NaN,NaN
12551,OWID_ASI,NaN,아시아,2022-10-03,185738973.0,84967.0,128301.714,1485943.0,205.0,284.429,...,NaN,NaN,NaN,NaN,NaN,4.721383e+09,NaN,NaN,NaN,NaN
208839,KOR,Asia,한국,2022-10-03,24848184.0,16423.0,24931.571,28528.0,19.0,36.571,...,40.9,NaN,12.27,83.03,0.916,5.181581e+07,NaN,NaN,NaN,NaN


In [11]:
## 3. df_covid19_100을 한국과 각 대륙별 열로 배치한 넓은(?) 형태의 데이터프레임으로 변환
df_covid19_100_wide = df_covid19_100.loc[:,['date', 'location', 'new_cases', 
'people_fully_vaccinated_per_hundred']].rename(columns={'new_cases':'확진자', 
'people_fully_vaccinated_per_hundred':'백신접종완료자'})

df_covid19_100_wide.head()

,date,location,확진자,백신접종완료자
2016,2022-10-03,아프리카,754.0,22.76
163005,2022-10-03,북미,45451.0,64.43
207753,2022-10-03,남미,15450.0,76.48
12551,2022-10-03,아시아,84967.0,71.77
208839,2022-10-03,한국,16423.0,85.73


In [12]:
# 위의 CategoricalDtype의 이유는 여기 때문인 거 같음

df_covid19_100_wide = df_covid19_100_wide.pivot(index='date', columns='location', 
values=['확진자', '백신접종완료자']).sort_values(by = 'date')

df_covid19_100_wide.head()

확진자                                                         \
location         한국       아시아        유럽       북미       남미    아프리카    오세아니아   
date                                                                         
2022-10-03  16423.0   84967.0  168183.0  45451.0  15450.0   754.0   5145.0   
2022-10-04  34710.0  149537.0  384493.0  61078.0   2379.0  1217.0   1536.0   
2022-10-05  28603.0  149638.0  330710.0  80045.0  18412.0   780.0   3098.0   
2022-10-06  22259.0  127116.0  394838.0  71088.0  14690.0   904.0  31919.0   
2022-10-07  19379.0  124355.0  375601.0  67185.0   4412.0   533.0   2517.0   

           백신접종완료자                                            
location        한국    아시아     유럽     북미     남미   아프리카  오세아니아  
date                                                          
2022-10-03   85.73  71.77  66.58  64.43  76.48  22.76  61.67  
2022-10-04   85.73  71.78  66.62  64.44  76.48  22.76  61.67  
2022-10-05   85.73  71.80  66.63  64.45  76.48  22.76  61.68  
2022-10-06   85.73  71.80  66.63  64.46  76.48  22.76  61.68  
2022-10-07   85.73  71.81  66.63  64.79  76.53  22.76  61.68

In [13]:
df_covid19_100_wide.columns = ['확진자_한국', '확진자_아시아', '확진자_유럽', '확진자_북미',  
'확진자_남미', '확진자_아프리카', '확진자_오세아니아', '백신접종완료자_한국', 
'백신접종완료자_아시아', '백신접종완료자_유럽', '백신접종완료자_북미', 
'백신접종완료자_남미', '백신접종완료자_아프리카', '백신접종완료자_오세아니아']

df_covid19_100_wide.head()

,확진자_한국,확진자_아시아,확진자_유럽,확진자_북미,확진자_남미,확진자_아프리카,확진자_오세아니아,백신접종완료자_한국,백신접종완료자_아시아,백신접종완료자_유럽,백신접종완료자_북미,백신접종완료자_남미,백신접종완료자_아프리카,백신접종완료자_오세아니아
date,,,,,,,,,,,,,,
2022-10-03,16423.0,84967.0,168183.0,45451.0,15450.0,754.0,5145.0,85.73,71.77,66.58,64.43,76.48,22.76,61.67
2022-10-04,34710.0,149537.0,384493.0,61078.0,2379.0,1217.0,1536.0,85.73,71.78,66.62,64.44,76.48,22.76,61.67
2022-10-05,28603.0,149638.0,330710.0,80045.0,18412.0,780.0,3098.0,85.73,71.80,66.63,64.45,76.48,22.76,61.68
2022-10-06,22259.0,127116.0,394838.0,71088.0,14690.0,904.0,31919.0,85.73,71.80,66.63,64.46,76.48,22.76,61.68
2022-10-07,19379.0,124355.0,375601.0,67185.0,4412.0,533.0,2517.0,85.73,71.81,66.63,64.79,76.53,22.76,61.68


In [14]:
## 4. covid19 데이터를 국가별로 종합 요약한 df_covid19_stat 생성

df_covid19_stat = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=True).agg(
    인구수 = ('population', 'max'), 
    전체사망자수 = ('new_deaths', 'sum'), 
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'), 
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'), 
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')).reset_index()

df_covid19_stat.head()

,iso_code,continent,location,인구수,전체사망자수,백신접종자완료자수,인구백명당백신접종완료율,인구백명당부스터접종자수
0,ABW,North America,Aruba,106459.0,340.0,84017.0,78.92,NaN
1,AFG,Asia,Afghanistan,41128772.0,7854.0,10825004.0,26.32,NaN
2,AGO,Africa,Angola,35588996.0,1933.0,8253873.0,23.19,4.09
3,AIA,North America,Anguilla,15877.0,13.0,10377.0,65.36,20.43
4,ALB,Europe,Albania,2842318.0,3596.0,1270016.0,44.68,13.27


In [15]:
df_covid19_stat['십만명당사망자수'] = round(df_covid19_stat['전체사망자수'] / 
                                    df_covid19_stat['인구수'] * 100000, 5)

df_covid19_stat['백신접종완료율'] = df_covid19_stat['백신접종자완료자수'] / df_covid19_stat['인구수']

df_covid19_stat.head()

,iso_code,continent,location,인구수,전체사망자수,백신접종자완료자수,인구백명당백신접종완료율,인구백명당부스터접종자수,십만명당사망자수,백신접종완료율
0,ABW,North America,Aruba,106459.0,340.0,84017.0,78.92,NaN,319.37178,0.789196
1,AFG,Asia,Afghanistan,41128772.0,7854.0,10825004.0,26.32,NaN,19.09612,0.263198
2,AGO,Africa,Angola,35588996.0,1933.0,8253873.0,23.19,4.09,5.43145,0.231922
3,AIA,North America,Anguilla,15877.0,13.0,10377.0,65.36,20.43,81.87945,0.653587
4,ALB,Europe,Albania,2842318.0,3596.0,1270016.0,44.68,13.27,126.51646,0.446824


In [16]:
## 여백 설정을 위한 변수 설정
margins_P = {'t' : 50, 'b' : 25, 'l' : 25, 'r' : 25}

In [17]:
# 데이터 로딩
df_employ_rate = pd.read_excel('2021_employment_rate.xlsx',\
                              # '학과별' 시트의 데이터를 불러오는데,
                              sheet_name = '학과별',\
                              # 앞의 13행은 제외하고
                              skiprows=(13),\
                              # 첫 번째 행은 열 이름으로 설정
                              header = 0)
# df_employ_rate에서 첫 번째부터 9번째까지의 열과 '계'로 끝나는 열을 선택하여 다시 df_employ_rate에 저장
df_employ_rate = pd.concat([df_employ_rate.iloc[:,0:8],
                           df_employ_rate.loc[:,df_employ_rate.columns.str.endswith('계')],\
                           df_employ_rate.loc[:,'입대자']],
                          axis = 1)

df_employ_rate.head()

,조사기준일,학제,과정구분,대계열,중계열,소계열,학과코드,학과명,졸업자_계,취업률_계,...,1차 유지취업자_계,1차 유지취업률_계,2차 유지취업자_계,2차 유지취업률_계,3차 유지취업자_계,3차 유지취업률_계,4차 유지취업자_계,4차 유지취업률_계,입학당시 기취업자_계,입대자
0,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100002,관광일본어과,79,29.6,...,10,76.9,8,61.5,7,53.8,5,38.5,0,14
1,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100003,관광일본어전공,26,61.1,...,7,87.5,7,87.5,7,87.5,7,87.5,1,0
2,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100005,관광일어과,107,50.0,...,34,81.0,32,76.2,27,64.3,25,59.5,4,3
3,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100011,실무일본어과,109,52.3,...,29,72.5,27,67.5,24,60.0,21,52.5,4,0
4,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100013,일본어과,63,46.2,...,14,70.0,13,65.0,8,40.0,8,40.0,4,4


In [18]:
# df_employ_rate에서 졸업자가 500명 이하인 학과 중 25% 샘플링
df_employ_rate_500 = df_employ_rate.loc[(df_employ_rate['졸업자_계'] < 500)]
df_employ_rate_500 = df_employ_rate_500.iloc[range(0, len(df_employ_rate_500.index), 4)] 
# -> 행 4개중 하나만 고르면서 넘어가기 때문에 결과적으로 25%가 샘플링 됨

In [19]:
df_employ_rate_500 = df_employ_rate_500.rename(columns = {'졸업자_계':'졸업자수', '취업률_계':'취업률', \
                                                          '취업자_합계_계':'취업자수'})

df_employ_rate_500.head()

,조사기준일,학제,과정구분,대계열,중계열,소계열,학과코드,학과명,졸업자수,취업률,...,1차 유지취업자_계,1차 유지취업률_계,2차 유지취업자_계,2차 유지취업률_계,3차 유지취업자_계,3차 유지취업률_계,4차 유지취업자_계,4차 유지취업률_계,입학당시 기취업자_계,입대자
0,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100002,관광일본어과,79,29.6,...,10,76.9,8,61.5,7,53.8,5,38.5,0,14
4,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100013,일본어과,63,46.2,...,14,70.0,13,65.0,8,40.0,8,40.0,4,4
8,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100049,항공호텔관광학부 일본어통역전공,40,37.5,...,5,62.5,5,62.5,5,62.5,5,62.5,0,2
12,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,중국어,C01010200003,관광중국어과,225,51.9,...,68,84.0,62,76.5,53,65.4,52,64.2,6,9
16,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,중국어,C01010200011,중국어과,5,60.0,...,3,100.0,2,66.7,2,66.7,2,66.7,0,0


,조사기준일,학제,과정구분,대계열,중계열,소계열,학과코드,학과명,졸업자수,취업률,...,1차 유지취업자_계,1차 유지취업률_계,2차 유지취업자_계,2차 유지취업률_계,3차 유지취업자_계,3차 유지취업률_계,4차 유지취업자_계,4차 유지취업률_계,입학당시 기취업자_계,입대자
0,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100002,관광일본어과,79,29.6,...,10,76.9,8,61.5,7,53.8,5,38.5,0,14
4,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100013,일본어과,63,46.2,...,14,70.0,13,65.0,8,40.0,8,40.0,4,4
8,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100049,항공호텔관광학부 일본어통역전공,40,37.5,...,5,62.5,5,62.5,5,62.5,5,62.5,0,2
12,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,중국어,C01010200003,관광중국어과,225,51.9,...,68,84.0,62,76.5,53,65.4,52,64.2,6,9
16,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,중국어,C01010200011,중국어과,5,60.0,...,3,100.0,2,66.7,2,66.7,2,66.7,0,0
73,2021.12.31,전문대학,전문대학과정,사회계열,경영ㆍ경제,경영ㆍ경제,C02010100025,관광경영과,493,64.0,...,173,83.2,160,76.9,137,65.9,129,62.0,21,10
77,2021.12.31,전문대학,전문대학과정,사회계열,경영ㆍ경제,경영ㆍ경제,C02010100072,마케팅경영과,78,56.9,...,29,96.7,24,80.0,21,70.0,21,70.0,2,5
81,2021.12.31,전문대학,전문대학과정,사회계열,경영ㆍ경제,경영ㆍ경제,C02010100092,부동산과,56,71.7,...,24,92.3,22,84.6,22,84.6,22,84.6,15,2
85,2021.12.31,전문대학,전문대학과정,사회계열,경영ㆍ경제,경영ㆍ경제,C02010100123,상경계열,91,78.6,...,38,88.4,37,86.0,35,81.4,34,79.1,17,4
89,2021.12.31,전문대학,전문대학과정,사회계열,경영ㆍ경제,경영ㆍ경제,C02010100154,인터넷비즈니스과,47,69.7,...,22,95.7,22,95.7,21,91.3,21,91.3,18,4
